<a href="https://colab.research.google.com/drive/1uDUdwJ3H841rV6Mb8D-1jg3VLhRlRSP7" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das dependências e configuração do ambiente

In [ ]:
# Instalação do PySpark e importação das bibliotecas
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, IntegerType, LongType
from pyspark.sql import functions as f
from google.colab import drive
import os
import zipfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Criação da seção
spark = SparkSession.builder\
     .master('local[*]')\
    .appName("analise_dados")\
    .config("spark.ui.port", '4050')\
    .getOrCreate()

In [ ]:
# Conexão com o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carregando os dados


In [ ]:
# Extraindo o zip para uma pasta do Drive
zipfile.ZipFile("/content/drive/MyDrive/desafio_datascience/semana-1.zip", "r").extractall("/content/drive/MyDrive/desafio_datascience/semana-1/")

In [ ]:
# Leitura do arquivo
path = "/content/drive/MyDrive/desafio_datascience/semana-1/dataset_bruto.json"
dados = spark.read.json(path)

# Exploração dos dados

In [ ]:
print(f"Quantidade de linhas do datafame: {dados.count()}")

Quantidade de linhas do datafame: 89083


In [ ]:
# Exibição das 5 primeiras linhas do dataframe
dados.limit(5).show(truncate= False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

In [ ]:
# Schema das colunas
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

## Criando um dataframe separado com as informações da coluna anuncio

In [ ]:
df1 = dados\
		.select(dados.anuncio.id.alias("id_join"),"anuncio.*")\
    .drop("endereco", "valores")
  
df2 = dados\
      .select(dados.anuncio.id.alias("id_join"), "anuncio.endereco.*","anuncio.valores.condominio", "anuncio.valores.iptu", "anuncio.valores.tipo", "anuncio.valores.valor")

df_anuncio = df1.join(df2, on=["id_join"], how="outer")
df_anuncio = df_anuncio.drop("id_join")
df_anuncio.show(n=5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+------+-------+---------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|condominio|  iptu|   tipo|    valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+------+-------+---------+
|    0|     [410]|    [410]|      [3]|          [Elevador]|00012605-9cae-45b...|    [4]|   [1]|       Usado| Apartamen

In [ ]:
# Schema das colunas
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |--

## Frequência e filtro das coluna tipo_uso, tipo_unidade e tipo_anuncio

In [ ]:
# Frequência da coluna tipo_uso
df_anuncio\
  .select("tipo_uso")\
  .groupBy("tipo_uso")\
  .agg(
      f.count("tipo_uso").alias("frequencia"))\
  .show() 

+-----------+----------+
|   tipo_uso|frequencia|
+-----------+----------+
|  Comercial|      4542|
|Residencial|     84541|
+-----------+----------+



In [ ]:
comercial = 4542
residencial = 84541
total = comercial + residencial

print(f"Comercial: {(comercial / total)*100:.2f}%, Residencial: {(residencial / total)*100:.2f}%")

Comercial: 5.10%, Residencial: 94.90%


- É possivel observar que a base possui mais imóveis do tipo Residencial (94.90%), enquanto o tipo Comercial equivale a 5.10% dos dados

In [ ]:
# Frequência da coluna tipo_unidade
df_anuncio\
  .select("tipo_unidade")\
  .groupBy("tipo_unidade")\
  .agg(
      f.count("tipo_unidade").alias("Frequência")
      )\
  .show()

+------------+----------+
|tipo_unidade|Frequência|
+------------+----------+
|      Outros|     11963|
| Apartamento|     66801|
|        Casa|     10319|
+------------+----------+



In [ ]:
outros = 11963
apartamento = 66801
casa = 10319
total = outros + apartamento + casa

print(f"Outros: {(outros / total)*100:.2f}%, Apartamento: {(apartamento / total)*100:.2f}%, Casa: {(casa / total)*100:.2f}%")

Outros: 13.43%, Apartamento: 74.99%, Casa: 11.58%


- O tipo de unidade que mais aparece é Apartamento (74.99%), a base apresenta mais dados com o tipo "Outros" (13.43%) do que com o tipo "Casa" (11.58%)

In [ ]:
# Frequência da coluna tipo_anuncio
df_anuncio\
  .select("tipo_anuncio")\
  .groupBy("tipo_anuncio")\
  .agg(
      f.count("tipo_anuncio").alias("Frequência")
      )\
  .show()

+------------+----------+
|tipo_anuncio|Frequência|
+------------+----------+
|       Usado|     88827|
|  Lançamento|       256|
+------------+----------+



In [ ]:
usado = 88827
lançamento = 256
total = usado + lançamento

print(f"Usado: {(usado / total)*100:.2f}%, Lançamento: {(lançamento / total)*100:.2f}%")

Usado: 99.71%, Lançamento: 0.29%


- 99.71% dos tipos dos anuncios são de imóveis usados, enquanto imóveis em lançamento correspondem a 0.29%

In [ ]:
# Filtrando a base procurando pelos tipos residencial, apartamento e usado
df_filtro = df_anuncio\
  .where(f.upper(df_anuncio.tipo_uso).like("RESIDENCIAL"))\
  .where(f.upper(df_anuncio.tipo_unidade).like("APARTAMENTO"))\
  .where(f.upper(df_anuncio.tipo_anuncio).like("USADO"))

df_filtro.show(n=5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|   bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|condominio|  iptu|   tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+------+-------+-------+
|    3|      [43]|     [43]|      [1]|[Churrasqueira, P...|a2e6d7a5-0ff0-484...|    [2]|   [0]|       Usado| Apartamento|Residencial| [0]|   Cosmos|23066271|

## Trabalhando com o DataFrame filtrado

- Optei por utilizar apenas os dados que se encontravam na primeira posição da lista em cada coluna

In [ ]:
print("ANTES DO TRATAMENTO")
df_filtro.printSchema()

df_filtro = df_filtro\
  .withColumn("quartos", df_filtro.quartos[0].cast(IntegerType()))\
  .withColumn("suites", df_filtro.suites[0].cast(IntegerType()))\
  .withColumn("banheiros", df_filtro.banheiros[0].cast(IntegerType()))\
  .withColumn("vaga", df_filtro.vaga[0].cast(IntegerType()))\
  .withColumn("area_total", df_filtro.area_total[0].cast(IntegerType()))\
  .withColumn("area_util", df_filtro.area_util[0].cast(IntegerType()))\

print("DEPOIS DO TRATAMENTO")
df_filtro.printSchema()

ANTES DO TRATAMENTO
root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (n

In [ ]:
df_filtro.show(n=5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|   bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|condominio|  iptu|   tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+------+-------+-------+
|    3|        43|       43|        1|[Churrasqueira, P...|a2e6d7a5-0ff0-484...|      2|     0|       Usado| Apartamento|Residencial|   0|   Cosmos|23066271|

### Informações sobre endereço (dados filtrados)

In [ ]:
df_localizacao = df_filtro\
                .select("bairro", "zona")
df_localizacao.show(truncate= False)

+------------------------+----------+
|bairro                  |zona      |
+------------------------+----------+
|Todos os Santos         |Zona Norte|
|Recreio dos Bandeirantes|Zona Oeste|
|Guaratiba               |Zona Oeste|
|Cosmos                  |Zona Oeste|
|Paciência               |Zona Oeste|
|Realengo                |Zona Oeste|
|Paciência               |Zona Oeste|
|Pedra de Guaratiba      |Zona Oeste|
|Paciência               |Zona Oeste|
|Guaratiba               |Zona Oeste|
|Guaratiba               |Zona Oeste|
|Cosmos                  |Zona Oeste|
|Guaratiba               |Zona Oeste|
|Paciência               |Zona Oeste|
|Santa Cruz              |Zona Oeste|
|Paciência               |Zona Oeste|
|Cosmos                  |Zona Oeste|
|Guaratiba               |Zona Oeste|
|Cosmos                  |Zona Oeste|
|Campo Grande            |Zona Oeste|
+------------------------+----------+
only showing top 20 rows



### Informações sobre valores (dados filtrados)

In [ ]:
print("ANTES DO TRATAMENTO")
df_filtro.printSchema()

df_filtro = df_filtro\
              .withColumn("condominio", df_filtro.condominio[0].cast(IntegerType()))\
              .withColumn("iptu", df_filtro.iptu[0].cast(IntegerType()))\
              .withColumn("tipo", df_filtro.tipo[0])\
              .withColumn("valor", df_filtro.valor[0].cast(IntegerType()))

print("DEPOIS DO TRATAMENTO")
df_filtro.printSchema()            

ANTES DO TRATAMENTO
root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pais: string (nullable = true)
 |-- rua: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- iptu: arra

In [ ]:
# Filtrando por "Venda" na coluna tipo
df_filtro = df_filtro\
  .where("tipo = 'Venda'")

df_filtro.show(n=5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+-----------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+----+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|     bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|condominio|iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+-----------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----------+----+-----+-------+
|    0|       410|      410|        3|          [Elevador]|00012605-9cae-45b...|      4|     1|       Usado| Apartamento|Residencial|   1|    Ipanema|22411000|Rio 

## Salvando Arquivo


In [ ]:
df_filtro.write.parquet(
    path="/content/drive/MyDrive/desafio_datascience/semana-1/parquet/base_tratada",
    mode= "overwrite"
)